# Mapping Dead Reckoning course data

Hello and welcome to this notebook. 

## Introduction

#### Aims

There are three key aims of this notebook:
- Gain some understanding of the Folium plugin.
- Deal with code that isn't behaving as expected.
- Map some ship course data, including showing how we can convert data to multiple courses.  

#### Data 

In a previous notebook, we downloaded some data from Discovery while investigating how to use the Discovery API. Here, we are going to use the images we downloaded - the ships logs from the 1800's. 

#### Starting imports

We start by importing the libraries we will need. We are going to assume that you don't have the libraries installed, so we will install them as we go.

In [ ]:
%pip install geopy
%pip install folium
import folium
import geopy
from geopy.distance import geodesic
from datetime import datetime, timedelta
import folium.plugins as plugins

### Dead reckoning

Prior to precision navigation systems, such as GPS, ships navigated using a techniquie called dead reckoning. This involves working from the last "known good" location of the ship, then using the speed and direction traveled to determine the current position. 
Tools such as sextants would have helped decrease the error, but speed and time were still valuable variables; thus they were recorded in detail and with care in the ships logs. Its these data we are going to use here. 

#### Heading data. 

In the 1800's, ships navigation was less accurate than can be achived with the modern 3-digit compass. Instead, they used a [32 wind compass rose](https://en.wikipedia.org/wiki/Points_of_the_compass#32-wind_compass_rose). 

If we investigate the logs we've downloaded, they will contain headings such as "ENE" or "NW". These are about as accurate as ships could navigate at this point, and represent a range of headings of 11.25 degrees, which we will need to compensate for. 

Helpfully, we can make an assumption about these ranges, making the amount of data we need to generate more manageable. As we have to assume that the data in the log are accurate, we can assume that the ship was anywhere within the range of headings - for example, for EbN (East by North), this would be 78.75 to 90 degrees. While the ship may have followed precisely the middle of these headings for the entire leg of the journey, it is far more lilkey that they would have drifted slightly of course. This could have been minor due to waves, or major due to currents or innacurate navigation. Thus, we have to assume that the ship could have been anywhere within the range of headings.

Thus, rather than generating 1 data point for each heading, we will generate 3. At the end of each leg of the course, this will give a range of positions that the ship could be in.

#### Sighting land

This technique of estimating three points for the entire journey isn't perfect; once the ship sighted land, they would have been able to take an accurate location. In reality, this would have fixed any drift from the dead reckoning. However, to keep this Notebook approachable, we are going to assume that the ship didn't sight land until the end of the journey. 

## Entering the data

#### Creating the lists

The first step is to create the lists that we will use to store the data. We will create a list for the left-most, middle and right-most data points. 

In [ ]:
center_course = []
left_course = []
right_course = []

#### The first point

We need to start with known good location - this will provide the initial lat and long that the rest of the course is built from. So first, we need to ask the user for this information. 

When the user runs this cell, they will be asked for this lat, long, and start date. This will then be used as the first waypoint for each of the three lists.

Note the format of the date - this helps us do some calculations later.

In [ ]:
latitude = float(input("Enter the latitude: "))
longitude = float(input("Enter the longitude: "))
start_date = input("Enter the start date (in the format yyyy-mm-dd): ")

# Center Course

center_course.append(
    {
        "latitude": latitude,
        "longitude": longitude,
        "date": start_date
    }
)

# Left Course

left_course.append(
    {
        "latitude": latitude,
        "longitude": longitude,
        "date": start_date
    }
)

# Right Course

right_course.append(
    {
        "latitude": latitude,
        "longitude": longitude,
        "date": start_date
    }
)

#### The next points

After the known good point, all we know is the bearing speed and time. So, we ask the user for this information, and use it to work out the next point. 

If the world was flat, this would be some simple trigonometry - we know the bearing, and we know the speed and time so we can work out the distance. We can then use these data to work out the lat and long of the next point. 

The world, however, isn't flat. Luckily, the [geopy](https://geopy.readthedocs.io/en/stable/) library has a function that will do this for us - geodesic. This takes the start, bearing, and distance, and returns the lat and long of the end point.

Here, we start to do some additional work - we also need to work out the left and right points. For this, we are going to take the bearing that the user provided, and add and subtract 5.625 degrees, and work out the left and right points.

In [ ]:
## Note, this cell should be run once for every new speed or course change - this could mean as
## much as once per hour

bearing = float(input("Enter the bearing: "))
speed = float(input("Enter the speed: "))
time = float(input("Enter the time in hours: "))

kts_to_kph = 1.852

## Center Course

### Get the last position

last_position = center_course[-1]

### Calculate the new position

new_central_waypoint = geodesic(kilometers=(speed * kts_to_kph * time)).destination([last_position["latitude"], last_position["longitude"]], bearing)
new_central_waypoint = {
    "latitude": new_central_waypoint.latitude,
    "longitude": new_central_waypoint.longitude,
    "date": (datetime.strptime(last_position["date"], "%Y-%m-%d") + timedelta(hours=time)).strftime("%Y-%m-%d")
}

### Add the new position to the list

center_course.append(new_central_waypoint)


## Left Course

### Get the last position

last_position = left_course[-1]

### Calculate the new position

new_left_waypoint = geodesic(kilometers=(speed * kts_to_kph * time)).destination([last_position["latitude"], last_position["longitude"]], bearing - 5.625)
new_left_waypoint = {
    "latitude": new_left_waypoint.latitude,
    "longitude": new_left_waypoint.longitude,
    "date": (datetime.strptime(last_position["date"], "%Y-%m-%d") + timedelta(hours=time)).strftime("%Y-%m-%d")
}

### Add the new position to the list

left_course.append(new_left_waypoint)


## Right Course

### Get the last position

last_position = right_course[-1]

### Calculate the new position

new_right_waypoint = geodesic(kilometers=(speed * kts_to_kph * time)).destination([last_position["latitude"], last_position["longitude"]], bearing + 5.625)
new_right_waypoint = {
    "latitude": new_right_waypoint.latitude,
    "longitude": new_right_waypoint.longitude,
    "date": (datetime.strptime(last_position["date"], "%Y-%m-%d") + timedelta(hours=time)).strftime("%Y-%m-%d")
}

### Add the new position to the list

right_course.append(new_right_waypoint)

## Mapping the data

We are going to use the Folium plugin to map the data. This is a very powerful plugin, with a range of options.

#### Static maps

Folium is an additive tool; we can add multiple features as we desire. For this static map, we are going to start with two - a line, and a marker at each waypoint.

The line is going to use the PolyLine function, and will take a list of coordinates. We are going to draw a line for each of the lists, with the main one being bigger and bolder than the others.

For the points, we are going to use the Marker tool. 

We are going to start by creating an empty map. In this example we will create a world map by centring on 0,0 and using a zoom level of 2. If you know the limit of your data, feel free to experiment by changing these values. 

In [ ]:
world_map = folium.Map(location=[0, 0], zoom_start=2)

Next, we are going to start adding the features. We will start with the main line. We start by creating a temporary list, isolating the coordinates from the main list. For this, we loop through the main list, selecting the lat and long each time. 

In [ ]:
location_coordinates_center = []
location_coordinates_left = []
location_coordinates_right = []

for position in center_course:
    location_coordinates_center.append([position["latitude"], position["longitude"]])

for position in left_course:
    location_coordinates_left.append([position["latitude"], position["longitude"]])

for position in right_course:
    location_coordinates_right.append([position["latitude"], position["longitude"]])

 We then add this to the map using the PolyLine function. Here, we specify which list of coordiates to use as the first variable, then specify details of the line we would like to draw. 

In [ ]:
## Center Course

folium.PolyLine(
    location_coordinates_center,
    color="blue",
    weight=4,
    opacity=1
).add_to(world_map)

The left and right lines are created in the same way, using the left and right lists.  

In [ ]:
## Left Course

folium.PolyLine(
    location_coordinates_left,
    color="red",
    weight=1,
    opacity=1
).add_to(world_map)

## Right Course

folium.PolyLine(
    location_coordinates_right,
    color="green",
    weight=2,
    opacity=1
).add_to(world_map)

After this, we add the points. We don't need to create the temporary list this time, we can just cycle through the main list, extracting the data we need at each point. As with the lines, we specify which data we want the marker to be located at, then details about the marker.

In [ ]:
## Center Course

for position in center_course:
    folium.CircleMarker(
        location=[position["latitude"], position["longitude"]],
        popup = position["date"],
        radius=2,
        color="blue",
        fill=True,
        fill_color="blue"
    ).add_to(world_map)

## Left Course

for position in left_course:
    folium.CircleMarker(
        location=[position["latitude"], position["longitude"]],
        popup = position["date"],
        radius=2,
        color="red",
        fill=True,
        fill_color="red"
    ).add_to(world_map)

## Right Course

for position in right_course:
    folium.CircleMarker(
        location=[position["latitude"], position["longitude"]],
        popup = position["date"],
        radius=2,
        color="green",
        fill=True,
        fill_color="green"
    ).add_to(world_map)

Finally, we show the map again. This time, it will have the details we have been creating added to it.

In [ ]:
world_map

#### Other things we may want to add

If we investigate the Folium documentation, we can see there are a number of different features we may want to add to the map, depending on the information we have available, and what we want to show. In all the cases, the documentation shows what data are needed, and in what format. 

For example, the [boat marker](https://python-visualization.github.io/folium/latest/user_guide/plugins/boat_marker.html) takes a heading and location for the boat, a heading and speed for the wind, and colour for the marker. If these are avaiable in the raw data, then a similar process to the markers can be followed to add them to the map. The data would be input at the same time as the course and speed data, then isolated and added to the map in a loop.

If the course mapped covers the world, but has a lot of detail, we may want to add a [minimap](https://python-visualization.github.io/folium/plugins.html#folium.plugins.MiniMap) to the map. This will allow the user to zoom into the area of interest while maintaining the context of the rest of the map.

## Animating the data

As we have time and location, we can animate the data. This will give us a really nice way of visualising the data; it will give context on how fast the ship was travelling at any one point. 

Note: this area is where the code isn't behaving as expected. The next few cells are for advanced users only, and can be skipped - if you want to see the animation and are using the default data, run the cells, but don't worry about the code or details. 

One of the main changes between polyline and timestampedgeojson is the data required. For polyline we just needed a list of coordinates, for timestampedgeojson we need start and end coordinates and start and end times.
We already have this data, we just need to do some shuffling to get it into the right arrangement.

We start similar to before, by creating an empty list for the lines. 

In [ ]:
lines_central = []
lines_left = []
lines_right = []

We then loop through the list of waypoints. We are looking for every waypoint and its following one, and the time at each. These represent the start and end of each line and the data we need; from these we can create the animation.
These points include aditional information. The "type" marker is used to by the animation. Others, such "stroke" and "stroke-width" are used to control the appearance of the line.

In [ ]:
## Center Course

for i in range(len(center_course) - 1):
    lines_central.append(
        {
            "type": "Feature",
            "geometry": {
                "type": "LineString",
                "coordinates": [
                    [center_course[i]["longitude"], center_course[i]["latitude"]],
                    [center_course[i + 1]["longitude"], center_course[i + 1]["latitude"]]
                ]
            },
            "properties": {
                "times": [
                    center_course[i]["date"],
                    center_course[i + 1]["date"]
                ],
                "stroke": "#0000ff",
                "stroke-width": 4,
                "style": {
                    "color": "blue"
                }
            }
        }
    )

## Left Course

for i in range(len(left_course) - 1):
    lines_left.append(
        {
            "type": "Feature",
            "geometry": {
                "type": "LineString",
                "coordinates": [
                    [left_course[i]["longitude"], left_course[i]["latitude"]],
                    [left_course[i + 1]["longitude"], left_course[i + 1]["latitude"]]
                ]
            },
            "properties": {
                "times": [
                    left_course[i]["date"],
                    left_course[i + 1]["date"]
                ],
                "stroke": "#ff0000",
                "stroke-width": 1,
                "style": {
                    "color": "red"
                }
            }
        }
    )

## Right Course

for i in range(len(right_course) - 1):
    lines_right.append(
        {
            "type": "Feature",
            "geometry": {
                "type": "LineString",
                "coordinates": [
                    [right_course[i]["longitude"], right_course[i]["latitude"]],
                    [right_course[i + 1]["longitude"], right_course[i + 1]["latitude"]]
                ]
            },
            "properties": {
                "times": [
                    right_course[i]["date"],
                    right_course[i + 1]["date"]
                ],
                "stroke": "#00ff00",
                "stroke-width": 2,
                "style": {
                    "color": "green"
                }
            }
        }
    )

#### How to animate the data

Now we have the data in the right format arrangement, its a similar process to the static map. We provide the TimestampedGeoJson object with the data we created in the previous cell, and specify additional details such as whether we want the animation to loop, or the time per frame. 

Note: this specific cell is where the code isn't behaving as expected.

In [ ]:
animated_map = folium.Map(location=[0, 0], zoom_start=2)

## Center Course

plugins.TimestampedGeoJson(
    {
        "type": "FeatureCollection",
        "features": lines_central
    },
    period="PT1H",
    add_last_point=True
).add_to(animated_map)

## Left Course

plugins.TimestampedGeoJson(
    {
        "type": "FeatureCollection",
        "features": lines_left
    },
    period="PT1H",
    add_last_point=True
).add_to(animated_map)

## Right Course

plugins.TimestampedGeoJson(
    {
        "type": "FeatureCollection",
        "features": lines_right
    },
    period="PT1H",
    add_last_point=True
).add_to(animated_map)

## And display the map

animated_map

#### That didn't work, what happened?

As we can see, the animation didn't work. 

Typically, we would look at the error message and work out a solution from the information provided. However, here we just get the message "Time not available". 

So, we need to look on GitHub to see if anyone else has had this problem - the issues tab is a good place to start. Here, we find [this issue](https://github.com/socib/Leaflet.TimeDimension/issues/83), where another user has had the same problem. As part of the conversation on this issue, we find a link to [an issue on a different repository](https://github.com/socib/Leaflet.TimeDimension/pull/214), seemingly fixing the same issue. 

We can investigate further. We know that Folium uses the JavaScript library (Leaflet)[https://github.com/Leaflet/Leaflet], and searching the web for "Leaflet TimeStampedGeoJson" leads us to [this page](https://github.com/python-visualization/folium/blob/main/folium/plugins/timestamped_geo_json.py) showing the code for the plugin. From here, at [line 77](https://github.com/python-visualization/folium/blob/main/folium/plugins/timestamped_geo_json.py#L77), we can find our way to the original plugin code, [here](https://github.com/socib/Leaflet.TimeDimension). This is the same repository that was lined to, with the issue that has since been fixed. 

This is good news, as we have confirmed that the issue is in the plugin, and not in our code. We can also see that the plugin is still being maintained, and that the issue has been fixed. All that we're wating for is a new release of the plugin, and for Leaflet and Folium to update to the new version.

For us, this leaves us with two options:
- Wait for the new version to be released, and update our code when it is.
- We know that the issue is with dates before 1970, so we can change the dates we're working with. For this example, the dates we're working with are all in the 1800's, so we can add 200 years to all of the dates. 

#### Adding 200 years to the dates
Adding 200 years to the date is a simple process, and we can do this using the datetime library. We simply loop through the dates, adding 200 years to each one.

In [ ]:
## Days in 200 years

days_in_200_years = 365 * 200

## Center Course

for i in range (len(lines_central)):
    lines_central[i]["properties"]["times"] = [str(datetime.strptime(lines_central[i]["properties"]["times"][0], "%Y-%m-%d") + timedelta(days=days_in_200_years)), str(datetime.strptime(lines_central[i]["properties"]["times"][1], "%Y-%m-%d") + timedelta(days=days_in_200_years))]

## Left Course

for i in range (len(lines_left)):
    lines_left[i]["properties"]["times"] = [str(datetime.strptime(lines_left[i]["properties"]["times"][0], "%Y-%m-%d") + timedelta(days=days_in_200_years)), str(datetime.strptime(lines_left[i]["properties"]["times"][1], "%Y-%m-%d") + timedelta(days=days_in_200_years))]

## Right Course

for i in range (len(lines_right)):
    lines_right[i]["properties"]["times"] = [str(datetime.strptime(lines_right[i]["properties"]["times"][0], "%Y-%m-%d") + timedelta(days=days_in_200_years)), str(datetime.strptime(lines_right[i]["properties"]["times"][1], "%Y-%m-%d") + timedelta(days=days_in_200_years))]

#### Re-running the animation

Now that we have updated the dates to post 1970, we can re-run the animation.

In [ ]:
animated_map = folium.Map(location=[0, 0], zoom_start=2)

## Center Course

plugins.TimestampedGeoJson(
    {
        "type": "FeatureCollection",
        "features": lines_central
    },
    period="PT1H",
    add_last_point=True
).add_to(animated_map)

## Left Course

plugins.TimestampedGeoJson(
    {
        "type": "FeatureCollection",
        "features": lines_left
    },
    period="PT1H",
    add_last_point=True
).add_to(animated_map)

## Right Course

plugins.TimestampedGeoJson(
    {
        "type": "FeatureCollection",
        "features": lines_right
    },
    period="PT1H",
    add_last_point=True
).add_to(animated_map)

## And display the map

animated_map